In [1]:
import pandas as pd
import numpy as np

In [3]:
!pip install pyarrow

     |████████████████████████████████| 42.3 MB 82.0 MB/s eta 0:00:01     |███████████████████████████████ | 40.9 MB 82.0 MB/s eta 0:00:01


In [16]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [17]:
df.shape[1]

19

In [5]:
# Standard Deviation of the Duration
df['Duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds()/60

In [6]:
# Cutted df
cutted_df = df[(df['Duration']>=1) & (df['Duration']<=60)]

In [7]:
# One Hot Encoding
cutted_df['DOLocationID'] = cutted_df['DOLocationID'].astype(str)
cutted_df['PULocationID'] = cutted_df['PULocationID'].astype(str)

/tmp/ipykernel_13693/2928959519.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cutted_df['DOLocationID'] = cutted_df['DOLocationID'].astype(str)
/tmp/ipykernel_13693/2928959519.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cutted_df['PULocationID'] = cutted_df['PULocationID'].astype(str)


In [8]:
dicts = cutted_df[['PULocationID', 'DOLocationID']].to_dict(orient='records')

In [9]:
from sklearn.feature_extraction import DictVectorizer

In [10]:
# Start vectorize and fit to dicts
dv = DictVectorizer()
X_train = dv.fit_transform(dicts)

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

y_train = cutted_df['Duration'].values

# Train Model
model = LinearRegression()
model.fit(X_train, y_train)

# Get y_pred
y_pred_train = model.predict(X_train)

# Evaluate cost RMSE
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
print(f"RMSE: {rmse_train:.4f}")


RMSE: 7.6493


In [12]:
# Procesar los datos de febrero de la misma manera
df_feb['tpep_pickup_datetime'] = pd.to_datetime(df_feb['tpep_pickup_datetime'])
df_feb['tpep_dropoff_datetime'] = pd.to_datetime(df_feb['tpep_dropoff_datetime'])
df_feb['duration'] = (df_feb['tpep_dropoff_datetime'] - df_feb['tpep_pickup_datetime']).dt.total_seconds() / 60
df_feb_filtered = df_feb[(df_feb['duration'] >= 1) & (df_feb['duration'] <= 60)]

# Convertir los IDs a string
df_feb_filtered['PULocationID'] = df_feb_filtered['PULocationID'].astype(str)
df_feb_filtered['DOLocationID'] = df_feb_filtered['DOLocationID'].astype(str)

/tmp/ipykernel_13693/1172113821.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_feb_filtered['PULocationID'] = df_feb_filtered['PULocationID'].astype(str)
/tmp/ipykernel_13693/1172113821.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_feb_filtered['DOLocationID'] = df_feb_filtered['DOLocationID'].astype(str)


In [13]:
# Crear lista de diccionarios
dicts_val = df_feb_filtered[['PULocationID', 'DOLocationID']].to_dict(orient='records')

# Transformar los datos de validación
X_val = dv.transform(dicts_val)
y_val = df_feb_filtered['duration'].values

In [14]:
# Predecir en los datos de validación
y_pred_val = model.predict(X_val)

# Calcular el RMSE
rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
print(f"RMSE :{rmse_val:.4f} ")

RMSE :7.8118 
